In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.optimizers import RMSprop
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from tensorflow.keras.models import load_model
from keras.utils import get_custom_objects

In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Layer
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import mean_squared_error
from tensorflow.keras import backend as K
from keras.initializers import Initializer
from sklearn.cluster import KMeans



class RBFLayer(Layer):
    def __init__(self, units, gamma,initializer=None, **kwargs):
        super(RBFLayer, self).__init__(**kwargs)
        self.units = units
        self.gamma = gamma
        if not initializer:
            self.initializer = RandomUniform(0.0, 1.0)
        else:
            self.initializer = initializer
        super(RBFLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for the centers of the RBFs
        self.centers = self.add_weight(name='centers',
                                      shape=(self.units, input_shape[-1]),
                                      initializer='uniform',
                                      trainable=True)
        self.gamma = self.add_weight(name='gamma',
                                     shape=(self.units,),
                                     initializer=Constant(
                                         value=self.gamma),
                                     # initializer='ones',
                                     trainable=True)
        super(RBFLayer, self).build(input_shape)

    def call(self, x):

        C = K.expand_dims(self.centers)
        H = K.transpose(C-K.transpose(x))
        return K.exp(-self.gamma * K.sum(H**2, axis=1))


        # C = self.centers[np.newaxis, :, :]
        # X = x[:, np.newaxis, :]

        # diffnorm = K.sum((C-X)**2, axis=-1)
        # ret = K.exp( - self.betas * diffnorm)
        # return ret

    def compute_output_shape(self, input_shape):
        return input_shape[0], self.units


class InitCentersKMeans(Initializer):
    """ Initializer for initialization of centers of RBF network
        by clustering the given data set.
    # Arguments
        X: matrix, dataset
    """

    def __init__(self, X, n_clusters=20, max_iter=6000):
        self.X = X
        self.max_iter = max_iter

    def __call__(self, shape, dtype=None):
        assert shape[1] == self.X.shape[1]

        n_centers = shape[0]
        km = KMeans(n_clusters, max_iter, verbose=0)
        km.fit(self.X)
        return km.cluster_centers_    
        get_custom_objects().update({'RBFLayer': RBFLayer})

In [3]:
def shot_model(gamma_values):
    df = pd.read_csv('data/P.csv', delimiter=';')
    X = df.drop(columns=["GOAL","7MACC","9MACC","6MACC","WINGACC","PIVOTACC","FASTBREAKACC"])
    y = df.GOAL
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    for gamma in gamma_values:
        print(f"Training model with gamma = {gamma}")
        
        model = Sequential()
        model.add(Flatten(input_shape=(6,)))
        model.add(RBFLayer(6, gamma))
        model.add(Dense(1, activation='linear'))
        model.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=['mean_squared_error', 'mean_absolute_error'])
        
         # Custom callback to collect metrics for plotting
        class MetricsCallback(tf.keras.callbacks.Callback):
            def on_epoch_end(self, epoch, logs=None):
                self.mse_values.append(logs['mean_squared_error'])
                self.mae_values.append(logs['mean_absolute_error'])

            def on_train_begin(self, logs=None):
                self.mse_values = []
                self.mae_values = []

        
        metrics_callback = MetricsCallback()
        history = model.fit(X_train, y_train, batch_size=32, epochs=200, validation_data=(X_test, y_test), callbacks=[metrics_callback], verbose=0)

        plt.figure(figsize=(16, 6))
        plt.subplot(1, 2, 1)
        plt.plot(history.history['loss'], label='Training Loss')
        plt.plot(history.history['val_loss'], label='Validation Loss')
        plt.title('Model Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(history.epoch, metrics_callback.mse_values, label='Training MSE', color='blue')
        plt.plot(history.epoch, history.history['val_mean_squared_error'], label='Validation MSE', color='orange')
        plt.xlabel('Epoch')
        plt.ylabel('Mean Squared Error')
        plt.title('Accuracy')
        plt.legend()
        plt.grid(True)

        plt.tight_layout()
        plt.show()
        plt.figure(figsize=(16, 6))


        pred_test = model.predict(X_test)
        def mean_squared_error1(y_test, pred_test):
            squared_errors = [(true - pred)**2 for true, pred in zip(y_test, pred_test)]
            mse = sum(squared_errors) / len(y_test)
            return mse
    
        mse = mean_squared_error1(y_test, pred_test)
        r2 = r2_score(y_test, pred_test)
        mae = mean_absolute_error(y_test, pred_test)
        print("Manual MSE:", mse)
        print(f"Mean Absolute Error (MAE)",mae)  
        print(f"R2 Score",r2)

gamma_values = [0.1, 0.2, 0.3, 0.4,0.5,0.6,0.7,0.8,0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0]  # Modify this list as needed

shot_model( gamma_values)


Training model with gamma = 0.1


NameError: name 'RandomUniform' is not defined